# Multi-agent Collaboration for Financial Analysis


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from  dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"]


## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

c:\Users\MCS\anaconda3\envs\crewai_env\lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\MCS\anaconda3\envs\crewai_env\lib\site-packages\crewai_tools\tools\scrapegraph_scrape_tool\scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("website_url")
c:\Users\MCS\anaconda3\envs\crewai_env\lib\site-packages\crewai_tools\tools\selenium_scraping_tool\selenium_scraping_tool.py:26: Pydant

## Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bde5edb3-0a5f-425d-ba26-8aeaf20c6a07                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 830fe17a-ed1f-4a6b-a377-9ccc7d02b70e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 830fe17a-ed1f-4a6b-a377-9ccc7d02b70e
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


🚀 Crew: crew
└── 📋 Task: 830fe17a-ed1f-4a6b-a377-9ccc7d02b70e
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Data Analyst
## Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance

Apple Inc. (AAPL) Follow Compare Time to buy AAPL? 198.15 +7.73 +(4.06%) At close: 4:00:01 PM EDT 197.90 -0.25 (-0.13%) After hours: 4:15:26 PM EDT Time to buy AAPL? 1D 5D 5.19% 1M -10.27% 6M -12.92% YTD -20.87% 1Y 13.20% 5Y 195.76% All 154,284.96%

Key Events
Mountain
Advanced Chart
Loading Chart for AAPL
Previous Close 190.42
Open 186.10
Bid 197.24 x 5000
Ask 198.99 x 2000
Day's Range 186.06 - 199.54
52 Week Range 164.08 - 260.10
Volume 86,538,298
Avg. Volume 61,058,588
Market Cap (intraday) 2.977T
Beta (5Y Monthly) 1.26
PE Ratio (TTM) 31.45
EPS (TTM) 6.30
Earnings Date May 1, 2025
Forward Dividend & Yield 1.00 (0.53%)
Ex-Dividend Date Feb 10, 2025
1y Target Est 238.75

Apple Inc. Overview
Consumer Electronics / Technology
Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, 

🚀 Crew: crew
└── 📋 Task: 830fe17a-ed1f-4a6b-a377-9ccc7d02b70e
       Status: Executing Task...
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   ├── 🔧 Failed Search the internet with Serper (2)
    │   ├── 🔧 Used Read website content (1)
    │   └── 🔧 Using Delegate work to coworker (1)
    └── 🤖 Agent: Data Analyst
            Status: ✅ Completed
        └── 🔧 Used Read website content (2)

🚀 Crew: crew
└── 📋 Task: 830fe17a-ed1f-4a6b-a377-9ccc7d02b70e
       Status: Executing Task...
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   ├── 🔧 Failed Search the internet with Serper (2)
    │   ├── 🔧 Used Read website content (1)
    │   └── 🔧 Using Delegate work to coworker (1)
    └── 🤖 Agent: Data Analyst
            Status: ✅ Completed
        └── 🔧 Used Read website content (2)



# Agent: Crew Manager
## Thought: Thought: I have gathered detailed information about Apple Inc. stock (AAPL) from Yahoo Finance.
Thought: I need to analyze this data to identify trends and predict market movements for AAPL.
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Analyze market data for AAPL to identify trends and predict market movements\", \"context\": \"Detailed stock information for AAPL obtained from Yahoo Finance\", \"coworker\": \"Data Analyst\"}"
## Tool Output: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance

Apple Inc. (AAPL) Follow Compare Time to buy AAPL? 198.15 +7.73 +(4.06%) At close: 4:00:01 PM EDT 197.90 -0.25 (-0.13%) After hours: 4:15:26 PM EDT Time to buy AAPL? 1D 5D 5.19% 1M -10.27% 6M -12.92% YTD -20.87% 1Y 13.20% 5Y 195.76% All 154,284.96%

Key Events
Mountain
Advanced Chart
Loading Chart for AAPL
Previous Close 190.42
Open 186.10
Bid 197.24 x 5000
Ask 198.99 x 2000
Day's Range 186.06 - 199.54
52 Week Range 

🚀 Crew: crew
└── 📋 Task: 830fe17a-ed1f-4a6b-a377-9ccc7d02b70e
       Status: Executing Task...
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   ├── 🔧 Failed Search the internet with Serper (2)
    │   ├── 🔧 Used Read website content (1)
    │   └── 🔧 Using Delegate work to coworker (1)
    └── 🤖 Agent: Data Analyst
            Status: ✅ Completed
        ├── 🔧 Used Read website content (2)
        └── 🧠 Thinking...

In [ ]:
from IPython.display import Markdown
Markdown(result)